In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Create Pipeline

In [2]:
from PynPoint2 import Pypeline

pipeline = Pypeline("/Users/markusbonse/Desktop/Workplace_wv",
                    "/Users/markusbonse/Desktop",
                    "/Users/markusbonse/Desktop/Results_ADI_wv")

In [71]:
from PynPoint2 import Pypeline

pipeline = Pypeline("/Users/markusbonse/Desktop/Workplace_no_wv",
                    "/Users/markusbonse/Desktop",
                    "/Users/markusbonse/Desktop/Results_ADI_no_wv")

## Create Data Port

In [80]:
from PynPoint2.core import InputPort

data_port = InputPort("09_star_arr_shifted_01",
                     pipeline.m_data_storage)

In [3]:
from PynPoint2.core import InputPort

data_port = InputPort("10_wavelet_denoised_soft_01",
                     pipeline.m_data_storage)

## Get Planet line

In [51]:
import numpy as np
pos = [66, 48] # speckle
#pos = [67, 60] # e
#pos = [67, 60]

planet_line = data_port[:, pos[0], pos[1]]

## Detection Class

In [52]:
from PynPoint2.util import PeakDetection
peak_detection = PeakDetection(planet_line)

In [53]:
peak_detection.detect_peaks(max_peaks=1)
peak_detection.calc_peak_support()

In [54]:
peak_detection.plot_pdf()

max mode -2.89087408069
mean 12.4901597103
median 6.97906322913


In [55]:
print peak_detection.get_best_peak_mean()
print peak_detection.get_best_peak_mode()
print peak_detection.get_lowest_peak_mean()
print peak_detection.get_lowest_peak_mode()
print peak_detection.get_highest_peak_mean()
print peak_detection.get_highest_peak_mode()
print peak_detection.get_peak_accuray()

12.8520566658
-2.89087408069
12.8520566658
-2.89087408069
12.8520566658
-2.89087408069
0.996197563312
